In [1]:
import sys
sys.path.append('..') 
import os

import psycopg2
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta

# Main

## Data Loading

In [2]:
# """DataLoad"""
# from src.utils.config import get_config
# from src.db_ops.db_ops import PosgreOps

# # Load Config
# config = get_config()
# # Initialize database operator, this case posgre operator
# pgops = PosgreOps(config=config)
# ### Define the required parameters to load the data ###
# # -------------------------------------------------------- #
# months = 84
# table_name = "daas.epdk_petrol_province"
# # -------------------------------------------------------- #
# # Fetch data from database
# epdk_petrol_province_data = pgops.get_monthly_data(table_name=table_name,
#                                                 today=date.today(), months=months)
# # Close the connection after obtaining the data
# pgops.shutdown()

In [3]:
# Load Raw data
epdk_petrol_province_data = pd.read_csv('data/daas_raw_data.csv', index_col=[0])
epdk_petrol_province_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5904 entries, 0 to 5903
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            5904 non-null   object 
 1   province        5904 non-null   object 
 2   gasolene_types  5904 non-null   float64
 3   diesel_types    5904 non-null   float64
 4   fuel_oil_types  5890 non-null   float64
 5   aviation_fuels  5875 non-null   float64
 6   marine_fuels    5826 non-null   float64
 7   paraffin_oil    5838 non-null   float64
 8   other_types     5824 non-null   float64
 9   total           5904 non-null   float64
 10  percentage      5904 non-null   float64
dtypes: float64(9), object(2)
memory usage: 553.5+ KB


## Preprocessing

In [4]:
"""Preprocessing"""
from src.preprocessing.preprocess import PreprocessData

# Initialize preprocess object
prep = PreprocessData(data=epdk_petrol_province_data)
### Define parameters to preprocess ###
# -------------------------------------------------------- #
target_col_list = ["date","province","diesel_types"]
row_drop_dict = {"province": "Toplam"} 
# format_date_flag = True # True by default
col_rename_dict = {"diesel_types":"current_month_consumption"}
anomaly_col = "current_month_consumption" 
# -------------------------------------------------------- #
# Preprocess data with given parameters
preprocessed_df = prep.preprocess_data(target_col_list=target_col_list,
                                 row_drop_dict=row_drop_dict,
                                 col_rename_dict=col_rename_dict,
                                 anomaly_col=anomaly_col)


Missing Dates are:  [numpy.datetime64('2021-10-01T00:00:00.000000000')]


## Feature Engineering

In [5]:
"""FeatureEngineering"""
from src.feature_engineering.feat_eng import FeatureEngineering

# Initialize feature engineering object
feng = FeatureEngineering(data=preprocessed_df)
# Apply feature engineering to the preprocessed data
feature_engineered_df = feng.feature_engineering()

c:\Users\02485955\bireysel_proje\env38\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\02485955\bireysel_proje\env38\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\02485955\bireysel_proje\env38\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\02485955\bireysel_proje\env38\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible start

Index(['date', 'province', 'current_month_consumption', 'trend', 'yhat_lower',
       'yhat_upper', 'yhat'],
      dtype='object')


## Ultimate Train - Test Split

In [ ]:
from src.model_selection.data_preperation import DataPreperation as dp

# Train-Test split 
